## testing adjacency matrix for separating basal, obliques, tufts, trunk

In [1]:
import numpy as np
import os
import sys

import os
import sys
sys.path.append("../")
sys.path.append("../Modules/")
import pandas as pd

load segments

In [2]:
sim_directory = "/home/drfrbc/Neural-Modeling/scripts/2024-12-17-16-16-39-ZiaoSynapses_final_detailed_syn_dist_analysis/Complex_InhGmaxApic204_InhGmaxDend7.0_SomaGmax6.0_ExcGmax-1.0351_Np1000"
seg_data = pd.read_csv(os.path.join(sim_directory, "segment_data.csv"))

In [3]:
# Function to fix pseg ending with (1)
def replace_pseg_with_greatest_seg(df):
    for i, row in df.iterrows():
        if row['pseg'].endswith('(1)'):
            # Extract the section prefix (e.g., "L5PCtemplate[0].dend[0]")
            section_prefix = row['pseg'].rsplit('(', 1)[0]

            # Filter segments in the same section
            matching_segs = df['seg'][df['seg'].str.startswith(section_prefix)]

            # Find the greatest value before (1)
            max_seg = None
            max_value = -np.inf
            for seg in matching_segs:
                if seg.endswith('(1)'):
                    continue
                try:
                    value = float(seg.split('(')[-1].rstrip(')'))
                    if value > max_value:
                        max_value = value
                        max_seg = seg
                except ValueError:
                    continue

            # Replace pseg if a valid max_seg is found
            if max_seg:
                df.at[i, 'pseg'] = max_seg

    return df

In [4]:
# Replace NaN parent segments with a placeholder (optional, e.g., "No Parent")
seg_data['pseg'] = seg_data['pseg'].fillna('No Parent')

In [5]:
seg_data = replace_pseg_with_greatest_seg(seg_data)

In [6]:
# Values in pseg that are not in seg
not_in_seg = set(seg_data['pseg']) - set(seg_data['seg'])

print("Values in pseg but not in seg:")
print(not_in_seg)

Values in pseg but not in seg:
{'No Parent'}


In [39]:
len(seg_to_index)

2512

In [40]:
len(seg_data)

2511

In [43]:
# Get unique segments
segments = sorted(set(seg_data['seg']).union(seg_data['pseg']))

# Create a unique list of segments (seg + pseg) directly from seg_data
unique_segments = pd.concat([seg_data['seg'], seg_data['pseg']]).unique()
seg_to_index = {seg: idx for idx, seg in enumerate(unique_segments)}



len(seg_data)
len(seg_to_index)

# Initialize an adjacency matrix with size equal to the number of unique segments
n = len(unique_segments)
adjacency_matrix = np.zeros((n, n), dtype=int)

# Populate the adjacency matrix
for _, row in seg_data.iterrows():
    parent = row['pseg']
    child = row['seg']
    # Ensure valid indices and exclude "No Parent"
    if parent in seg_to_index and child in seg_to_index and parent != 'No Parent':
        parent_idx = seg_to_index[parent]
        child_idx = seg_to_index[child]
        adjacency_matrix[parent_idx, child_idx] = 1

In [8]:
from Modules import adjacency

In [45]:
soma_children = adjacency.get_children_indices(0, adjacency_matrix)

In [64]:
y_coords = []
for i, seg in seg_data.iterrows():
    # print(seg["p1_1"])
    y_coord = seg["p1_1"]#.iloc[0] if not seg["p1_1"] else None
    y_coords.append(y_coord)
#print(f"y_coords: {y_coords}")
#print(f"all_seg_list: {all_seg_list}")
apical_segment_indices = [i for i, seg in seg_data.iterrows() if 'apic' in seg.section]
#print(f"apical_segment_indices: {apical_segment_indices}")
nexus_index_in_all_list, _ = adjacency.find_branching_seg_with_most_branching_descendants_in_subset_y(adjacency_matrix, apical_segment_indices, y_coords)

In [74]:
seg_data[seg_data['Unnamed: 0'] == nexus_index_in_all_list]

,Unnamed: 0,section,idx_in_section_type,seg_half_seg_RA,L,seg,pseg,Section_L,Section_diam,Distance,...,pc_0,pc_1,pc_2,p1_0,p1_1,p1_2,r,dl_0,dl_1,dl_2
1475,1475,apic,36,0.397875,5.005411,L5PCtemplate[0].apic[36](0.989796),L5PCtemplate[0].apic[36](0.969388),245.265118,2.57,624.284848,...,18.948452,591.963829,-57.742575,18.73,594.349976,-57.02,1.415,-0.517102,4.846356,0.986031
